<a href="https://colab.research.google.com/github/AnuarKenzhibekov/Samruk_finetuned-xlm-roberta_0.1/blob/main/Samruk_finetuned-xlm-roberta_0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/google-bert/bert-base-multilingual-uncased

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google-bert/bert-base-multilingual-uncased)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="google-bert/bert-base-multilingual-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
-

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-uncased")

Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

encodings = tokenizer("Какова миссия Фонда?", truncation=True, padding="max_length", max_length=256)


In [6]:
import pandas as pd
from datasets import Dataset

train_df = pd.read_csv("train-02.csv")
test_df  = pd.read_csv("test-02.csv")

# HuggingFace Dataset
train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)


In [7]:
labels = sorted(train_df["label"].unique())
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}

def encode_batch(batch):
    enc = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    enc["labels"] = [label2id[l] for l in batch["label"]]
    return enc

train_ds = train_ds.map(encode_batch, batched=True)
test_ds  = test_ds.map(encode_batch, batched=True)

cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format(type="torch", columns=cols)
test_ds.set_format(type="torch", columns=cols)


Map:   0%|          | 0/6298 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [ ]:
import transformers
import numpy as np
import random
import torch
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

args = TrainingArguments(
    output_dir="./out",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=3,
)

def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "macro_precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "macro_recall": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "macro_f1": f1_score(y_true, y_pred, average="macro", zero_division=0),
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-479755135.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Currently logged in as: kenzhibekovanuar (kenzhibekovanuar-aitu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


In [13]:
trainer.evaluate(test_ds)


{'eval_loss': 0.7818223237991333,
 'eval_accuracy': 0.7742857142857142,
 'eval_macro_precision': 0.7783408709164671,
 'eval_macro_recall': 0.7720200132612849,
 'eval_macro_f1': 0.7716568519347592,
 'eval_runtime': 9.5247,
 'eval_samples_per_second': 73.493,
 'eval_steps_per_second': 9.239,
 'epoch': 10.0}

In [14]:
trainer.save_model("Samruk_finetuned-xlm-roberta_0.1")
tokenizer.save_pretrained("Samruk_finetuned-xlm-roberta_0.1")


('Samruk_finetuned-xlm-roberta_0.1/tokenizer_config.json',
 'Samruk_finetuned-xlm-roberta_0.1/special_tokens_map.json',
 'Samruk_finetuned-xlm-roberta_0.1/sentencepiece.bpe.model',
 'Samruk_finetuned-xlm-roberta_0.1/added_tokens.json',
 'Samruk_finetuned-xlm-roberta_0.1/tokenizer.json')

In [2]:
from transformers import pipeline

clf = pipeline("text-classification", model="Samruk_finetuned-xlm-roberta_0.1", tokenizer=tokenizer)

print(clf("Какова миссия Фонда?"))
print(clf("Қордың миссиясы қандай?"))
print(clf("What is the mission of the Fund?"))


NameError: name 'tokenizer' is not defined

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: Samruk_finetuned-xlm-roberta_0.1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`